## Import

In [22]:
from ipywidgets import *
from scipy.stats import zscore, pearsonr, kendalltau, spearmanr, ttest_rel, ttest_1samp, ttest_ind
from sklearn.decomposition import PCA
import numpy as np
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
import pickle
from scipy.ndimage import gaussian_filter1d
warnings.filterwarnings(action='ignore')

np.set_printoptions(suppress=True)

mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.bottom'] = False
mpl.rcParams['axes.spines.left'] = True

def cos_sim(A, B):
    return np.dot(A, B)/(np.linalg.norm(A)*np.linalg.norm(B))

In [2]:
exclude_short_movie = np.load('/home/dasom/State_dynamics/results/exclude_short_movie.npy')

rest_tr_ep1 = np.load('/home/dasom/State_dynamics/results/rest_tr_ep1.npy')
rest_tr_ep2 = np.load('/home/dasom/State_dynamics/results/rest_tr_ep2.npy')
rest_tr_ep3 = np.load('/home/dasom/State_dynamics/results/rest_tr_ep3.npy')

rest_total = rest_tr_ep1.shape[0] + rest_tr_ep2.shape[0] + rest_tr_ep3.shape[0]
rest_tr = np.hstack([rest_tr_ep1, rest_tr_ep2+996, rest_tr_ep3+996+958])
print(f'{rest_tr_ep1.shape[0]} + {rest_tr_ep2.shape[0]} + {rest_tr_ep3.shape[0]} = {rest_total}')

986 + 914 + 666 = 2566


## Novelty and Memorability measurements

#### Memorability

In [3]:
Memorability_score_total = np.load('/home/dasom/State_dynamics/results/Memorability_score_total.npy')
Memorability_score = np.load('/home/dasom/State_dynamics/results/Memorability_score.npy')
Memorability_score_binary = np.load('/home/dasom/State_dynamics/results/Memorability_score_binary.npy')
memorability = np.mean(Memorability_score, axis=0)

#### Novelty

In [4]:
cooccur_sim = np.load('/home/dasom/State_dynamics/results/cooccur_similarity.npy')

nove_cooccur_1 = np.where(cooccur_sim >= np.percentile(cooccur_sim,200/3))[0]
nove_cooccur_2 = np.where((cooccur_sim >= np.percentile(cooccur_sim,100/3)) & (cooccur_sim < np.percentile(cooccur_sim,200/3)))[0]
nove_cooccur_3 = np.where(cooccur_sim < np.percentile(cooccur_sim,100/3))[0]

valence_sim = np.load('/home/dasom/State_dynamics/results/valence_similarity.npy')

nove_valence_1 = np.where(valence_sim >= np.percentile(valence_sim,200/3))[0]
nove_valence_2 = np.where((valence_sim >= np.percentile(valence_sim,100/3)) & (valence_sim < np.percentile(valence_sim,200/3)))[0]
nove_valence_3 = np.where(valence_sim < np.percentile(valence_sim,100/3))[0]

### Just valence
valence = np.load('/home/dasom/State_dynamics/results/valence_just.npy')

#### Relationship novelty and memorability

In [ ]:
np.random.seed(2023)
fig = plt.figure(figsize = [10,5])
axs = fig.add_subplot(1,2,1)
subj_memory = np.array([np.mean(Memorability_score[:,i], axis=1) for i in [nove_cooccur_1, nove_cooccur_2, nove_cooccur_3]])
violin_parts = axs.violinplot(subj_memory.T, np.arange(3), showmeans=False, showmedians=False, showextrema=False, widths = 0.5)
for vp in violin_parts['bodies']:
    vp.set_facecolor('#FF0000')  # aa0000 FF00FF 00FFFF
    vp.set_edgecolor('#000000')
    vp.set_alpha(0.5)

for c in range(3):
    plt.scatter(np.random.normal(loc = c, scale = 0.02, size = 24), subj_memory[c], color = 'grey')
    plt.hlines(np.mean(subj_memory[c]), c-0.1, c+0.1, zorder = 1, color = 'k')

axs.set_xticks([0,1,2])
axs.set_yticks([0,0.1,0.2,0.3,0.4,0.5])
axs.set_yticklabels([0,0.1,0.2,0.3,0.4,0.5])
axs.set_xlim([-0.5,2.5])
axs.set_ylim([0, 0.5])

axs = fig.add_subplot(1,2,2)
subj_memory = np.array([np.mean(Memorability_score[:,i], axis=1) for i in [nove_valence_1, nove_valence_2, nove_valence_3]])
violin_parts = axs.violinplot(subj_memory.T, np.arange(3), showmeans=False, showmedians=False, showextrema=False, widths = 0.5)
for vp in violin_parts['bodies']:
    vp.set_facecolor('#9C27B0')  # aa0000 FF00FF 00FFFF
    vp.set_edgecolor('#000000')
    vp.set_alpha(0.5)

for c in range(3):
    plt.scatter(np.random.normal(loc = c, scale = 0.02, size = 24), subj_memory[c], color = 'grey')
    plt.hlines(np.mean(subj_memory[c]), c-0.1, c+0.1, zorder = 1, color = 'k')

axs.set_xticks([0,1,2])
axs.set_yticks([0,0.1,0.2,0.3,0.4,0.5])
axs.set_yticklabels([0,0.1,0.2,0.3,0.4,0.5])
axs.set_xlim([-0.5,2.5])
axs.set_ylim([0, 0.5])
fig.tight_layout()
fig.savefig('/home/dasom/Dropbox/ubuntu-window/미팅자료/Neural_dynamics/Memory_novelty_cooccur_valence/Relationship_btw_novelty_memory.png', dpi=300, bbox_inches = "tight")

In [ ]:
# score_mean = np.array([np.mean(Memorability_score[:,i], axis=1) for i in [nove_cooccur_1, nove_cooccur_2, nove_cooccur_3]])
score_mean = np.array([np.mean(Memorability_score[:,i], axis=1) for i in [nove_valence_1, nove_valence_2, nove_valence_3]])
ttest_rel(score_mean[0], score_mean[1])

#### Movie event onset and offset

In [5]:
overlap_start = np.load('/home/dasom/State_dynamics/results/movie_event_start_idx.npy')
overlap_end = np.load('/home/dasom/State_dynamics/results/movie_event_boundary_idx.npy')

#### Recall onset and offset

In [6]:
event_start_tot = np.load('/home/dasom/State_dynamics/results/Recall_event_onset.npy', allow_pickle=True)
event_end_tot = np.load('/home/dasom/State_dynamics/results/Recall_event_offset.npy', allow_pickle=True)
event_idx = np.load('/home/dasom/State_dynamics/results/Recall_event_idx.npy', allow_pickle=True)

## Brain data

In [9]:
with open('/media/dasom/caee0336-77a9-438c-af2f-7cf5b88293e7/dasom/SM_DATA/social_movie1/results/fMRI_data/movie_data_ex2_sc_sm.pickle', 'rb') as f:
    movie_data = pickle.load(f)
    
# with open('/media/dasom/caee0336-77a9-438c-af2f-7cf5b88293e7/dasom/SM_DATA/social_movie1/results/fMRI_data/recall_data_ex2_sc_sm.pickle', 'rb') as f:
#     recall_data_concat = pickle.load(f)

In [11]:
brain_mask = nib.load(f'/home/dasom/SM_DATA/mri_data/mask_files/MNI152NLin2009cAsym_3mm_mask.nii.gz').get_fdata()
mask = nib.load('/home/dasom/SM_DATA/mri_data/mask_files/BNA_3mm_atlas.nii.gz').get_fdata()

In [23]:
# movie viewing

window_length = 21

roi_mask1 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [215,216,217,218]]) # HIPP
roi_mask2 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [111,112,119,120]]) # PHC
roi_mask3 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [115,116]]) # EC
roi_mask4 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [117,118]]) # PRC
roi_mask5 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [75,76,79,80]]) # STG
roi_mask6 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [87,88,121,122]]) # STS
roi_mask7 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [143,144]]) # TPJ
roi_mask8 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [153,154,175,176]]) # PMC
roi_mask9 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [13,14]]) # DMPFC
roi_mask10 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [41,42]]) # VMPFC
roi_mask11 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [189,190,191,192,193,194]]) # VC
roi_mask12 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [71,72]]) # AC
roi_mask13 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [179,187,188]]) # ACC

patterns = []
# for rm in [roi_mask1,roi_mask2,roi_mask3,roi_mask4,roi_mask5,roi_mask6,roi_mask7,roi_mask8,roi_mask9,roi_mask10,roi_mask11,roi_mask12,roi_mask13]:
for rm in [roi_mask1]:
    pattern = []
    for s in range(movie_data.shape[0]):
        movie_data_sub = movie_data[s, rm]
        ev_pattern = []
        for ev in overlap_end[exclude_short_movie[1:]]:
            ev_pattern.append(movie_data_sub[:,ev-6:ev+15])
        pattern.append(ev_pattern)
    # patterns.append(np.array(pattern))
    patterns.append(gaussian_filter1d(pattern, sigma = 2, axis=-1)) # For visualization

In [214]:
# narrative recall
window_length = 21 # 18 for end
roi_mask1 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [215,216,217,218]]) # HIPP

patterns_recall = []
for rm in [roi_mask1]:
    pattern = []
    for s in range(len(recall_data_concat)):
        recall_data_sub = recall_data_concat[s][rm]
        ev_pattern = []
        for ev in event_start_tot[s]:
            ev_pattern.append(recall_data_sub[:,ev-6:ev+15])
        pattern.append(np.array(ev_pattern))
    patterns_recall.append(pattern)
    # patterns.append(gaussian_filter1d(pattern, sigma = 2, axis=-1)) # For visualization

#### Canonical space

In [ ]:
roi_mask1 = np.hstack([np.where(mask[brain_mask > 0] == r)[0] for r in [215,216,217,218]])
pca_canonical = PCA(n_components=12)
data = np.mean(movie_data[:,roi_mask1][:,:,rest_tr], axis=0)
data_hipp = pca_canonical.fit(data.T)

## Subspace analysis

### Linear regression

#### Novelty & Memory

In [13]:
which_cognition = 'memory'

np.random.seed(2023)

repetition = 10
beta_true = []
beta_perm = []
for data_pattern in patterns:
    beta_tot_true = []
    beta_tot_perm = []
    for repeat in range(repetition + 1):
        if repeat == 0:
            if which_cognition == 'novelty':
                conds = np.array([np.intersect1d(nove_cooccur_1, nove_valence_1), np.intersect1d(nove_cooccur_1, nove_valence_2), np.intersect1d(nove_cooccur_1, nove_valence_3),
                                  np.intersect1d(nove_cooccur_2, nove_valence_1), np.intersect1d(nove_cooccur_2, nove_valence_2), np.intersect1d(nove_cooccur_2, nove_valence_3),
                                  np.intersect1d(nove_cooccur_3, nove_valence_1), np.intersect1d(nove_cooccur_3, nove_valence_2), np.intersect1d(nove_cooccur_3, nove_valence_3)])
            elif which_cognition == 'valence':
                conds = np.array(np.array_split(np.argsort(valence), 6))
            elif which_cognition == 'memory':
                conds = np.array_split(np.argsort(-memorability), 9) # from low to high
            
            conds_regressor = []
            for cd in conds:
                cr = np.zeros(43)
                cr[cd] = 1
                conds_regressor.append(cr)

            FIR_tot = []
            for s in range(movie_data.shape[0]):
                F = np.array([conds_regressor[cr] for cr in range(len(conds_regressor))])
                beta = []
                for t in range(window_length):
                    beta.append(np.dot(np.dot(np.linalg.inv(np.dot(F, F.T)), F), data_pattern[s][:,:,t]))
                FIR_tot.append(beta)
            FIR_tot = np.array(FIR_tot)

            for s in range(movie_data.shape[0]):
                if which_cognition == 'novelty':
                    F = np.array([[-1,-1,-1,0,0,0,1,1,1],
                                  [-1,0,1,-1,0,1,-1,0,1],
                                  [1,1,1,1,1,1,1,1,1]]) 
                elif which_cognition == 'valence':
                    F = np.array([[-5,-3,-1,1,3,5],
                                  [1,1,1,1,1,1]])  
                elif which_cognition == 'memory':   
                    F = np.array([[np.mean(memorability[ev]) - np.mean(memorability) for ev in conds],
                                  [1,1,1,1,1,1,1,1,1]])             
                beta = []
                for t in range(window_length):
                    beta.append(np.dot(np.dot(np.linalg.inv(np.dot(F, F.T)), F), FIR_tot[s][t]))
                beta_tot_true.append(beta)    

        else:
            conds_regressor = []
            for cd in conds:
                cr = np.zeros(43)
                cr[cd] = 1
                conds_regressor.append(cr)

            FIR_tot = []
            for s in range(movie_data.shape[0]):
                F = np.array([conds_regressor[cr] for cr in range(len(conds_regressor))])
                beta = []
                for t in range(window_length):
                    beta.append(np.dot(np.dot(np.linalg.inv(np.dot(F, F.T)), F), data_pattern[s][:,:,t]))
                FIR_tot.append(beta)
            FIR_tot = np.array(FIR_tot)

            betas = []
            for s in range(movie_data.shape[0]):
                if which_cognition == 'novelty':
                    F = np.array([[-1,-1,-1,0,0,0,1,1,1],
                                  [-1,0,1,-1,0,1,-1,0,1],
                                  [1,1,1,1,1,1,1,1,1]]) 
                elif which_cognition == 'valence':
                    F = np.array([[-5,-3,-1,1,3,5],
                                  [1,1,1,1,1,1]])  
                elif which_cognition == 'memory':
                    F = np.array([[np.mean(memorability[ev]) - np.mean(memorability) for ev in conds],
                                  [1,1,1,1,1,1,1,1,1]])                          
                np.random.shuffle(F.T)
                beta = []
                for t in range(window_length):
                    beta.append(np.dot(np.dot(np.linalg.inv(np.dot(F, F.T)), F), FIR_tot[s][t]))
                betas.append(beta)
            beta_tot_perm.append(betas)
    beta_true.append(np.array(beta_tot_true))
    beta_perm.append(np.array(beta_tot_perm))

In [16]:
beta_true_novelty = beta_true
beta_perm_novelty = beta_perm

In [14]:
beta_true_memory = beta_true
beta_perm_memory = beta_perm

#### Retrieval

In [216]:
np.random.seed(2023)

repetition = 100
beta_true_recall = []
beta_perm_recall = []
for data_pattern in patterns_recall:
    beta_tot_true = []
    beta_tot_perm = []
    for repeat in range(repetition + 1):
        if repeat == 0:
            for s in range(movie_data.shape[0]):

                score_sub = Memorability_score_total[s][event_idx[s]]
                recall_task6 = np.where(score_sub >= np.percentile(score_sub, 500/6))[0]
                recall_task5 = np.where((score_sub >= np.percentile(score_sub, 400/6)) & (score_sub < np.percentile(score_sub, 500/6)))[0]
                recall_task4 = np.where((score_sub >= np.percentile(score_sub, 300/6)) & (score_sub < np.percentile(score_sub, 400/6)))[0]
                recall_task3 = np.where((score_sub >= np.percentile(score_sub, 200/6)) & (score_sub < np.percentile(score_sub, 300/6)))[0]
                recall_task2 = np.where((score_sub >= np.percentile(score_sub, 100/6)) & (score_sub < np.percentile(score_sub, 200/6)))[0]
                recall_task1 = np.where(score_sub < np.percentile(score_sub, 100/6))[0]

                conds = np.array([recall_task6, recall_task5, recall_task4, recall_task3, recall_task2, recall_task1])
                
                conds_regressor = []
                for cd in conds:
                    cr = np.zeros(len(score_sub))
                    cr[cd] = 1
                    conds_regressor.append(cr)
                    
                F = np.array([conds_regressor[cr] for cr in range(len(conds_regressor))])
                FIR_tot = []
                for t in range(window_length):
                    FIR_tot.append(np.dot(np.dot(np.linalg.inv(np.dot(F, F.T)), F), data_pattern[s][:,:,t]))
                FIR_tot = np.array(FIR_tot)

                F = np.array([[np.mean(score_sub[ev]) - np.mean(score_sub) for ev in conds],
                              [1,1,1,1,1,1]])                
                beta = []
                for t in range(window_length):
                    beta.append(np.dot(np.dot(np.linalg.inv(np.dot(F, F.T)), F), FIR_tot[t]))
                beta_tot_true.append(beta)    

        else:
            betas = []
            for s in range(movie_data.shape[0]):
                
                score_sub = Memorability_score_total[s][event_idx[s]]
                recall_task6 = np.where(score_sub >= np.percentile(score_sub, 500/6))[0]
                recall_task5 = np.where((score_sub >= np.percentile(score_sub, 400/6)) & (score_sub < np.percentile(score_sub, 500/6)))[0]
                recall_task4 = np.where((score_sub >= np.percentile(score_sub, 300/6)) & (score_sub < np.percentile(score_sub, 400/6)))[0]
                recall_task3 = np.where((score_sub >= np.percentile(score_sub, 200/6)) & (score_sub < np.percentile(score_sub, 300/6)))[0]
                recall_task2 = np.where((score_sub >= np.percentile(score_sub, 100/6)) & (score_sub < np.percentile(score_sub, 200/6)))[0]
                recall_task1 = np.where(score_sub < np.percentile(score_sub, 100/6))[0]

                conds = np.array([recall_task6, recall_task5, recall_task4, recall_task3, recall_task2, recall_task1])
                
                conds_regressor = []
                for cd in conds:
                    cr = np.zeros(len(score_sub))
                    cr[cd] = 1
                    conds_regressor.append(cr)

                F = np.array([conds_regressor[cr] for cr in range(len(conds_regressor))])
                FIR_tot = []
                for t in range(window_length):
                    FIR_tot.append(np.dot(np.dot(np.linalg.inv(np.dot(F, F.T)), F), data_pattern[s][:,:,t]))
                FIR_tot = np.array(FIR_tot)
                
                F = np.array([[np.mean(score_sub[ev]) - np.mean(score_sub) for ev in conds],
                              [1,1,1,1,1,1]])           
                np.random.shuffle(F.T)
                beta = []
                for t in range(window_length):
                    beta.append(np.dot(np.dot(np.linalg.inv(np.dot(F, F.T)), F), FIR_tot[t]))
                betas.append(beta)
            beta_tot_perm.append(betas)
    beta_true_recall.append(np.array(beta_tot_true))
    beta_perm_recall.append(np.array(beta_tot_perm))

In [217]:
beta_true_recall_start = beta_true_recall
beta_perm_recall_start = beta_perm_recall

In [ ]:
beta_true_recall_end = beta_true_recall
beta_perm_recall_end = beta_perm_recall

### PCA

#### projection

In [ ]:
FIR_tot_novelty_ev = np.load('/media/dasom/caee0336-77a9-438c-af2f-7cf5b88293e7/dasom/Neural_subspace/results/FIR_tot_novelty_ev.npy')
FIR_tot_memory_ev = np.load('/media/dasom/caee0336-77a9-438c-af2f-7cf5b88293e7/dasom/Neural_subspace/results/FIR_tot_memory_ev.npy')
FIR_tot_recall_ev = np.load('/media/dasom/caee0336-77a9-438c-af2f-7cf5b88293e7/dasom/Neural_subspace/results/FIR_tot_recall_start_ev.npy')

In [24]:
# memorability
F = []
for ev in np.array_split(np.argsort(-memorability), 9):
    a = np.zeros(43)
    a[ev] = 1 
    F.append(a)
F = np.array(F)

FIR_tot_memory_ev = []
for data_pattern in patterns:
    FIR_ev = []
    for s in range(movie_data.shape[0]):
        beta = []
        for t in range(window_length):
            beta.append(np.dot(np.dot(np.linalg.inv(np.dot(F, F.T)), F), data_pattern[s][:,:,t]))
        FIR_ev.append(beta)
    FIR_tot_memory_ev.append(np.array(FIR_ev))

In [25]:
fMRI_data = FIR_tot_memory_ev
pc_num = [window_length-1]*len(patterns)

pc1_evs = []
pc2_evs = []
p_c1_trues = []
p_c2_trues = []
p_c1_perms = []
p_c2_perms = []

for r in range(len(patterns)):

    beta_pcas = beta_true_memory[r]

    pca_c1 = PCA(n_components=pc_num[r])
    data = np.mean(beta_pcas, axis=0)[:,0].T
    data_c1 = pca_c1.fit(data.T)
    pcs_c1 = data_c1.components_
    pc1_evs.append(np.cumsum(pca_c1.explained_variance_ratio_))

    pca_c2 = PCA(n_components=pc_num[r])
    data = np.mean(beta_pcas, axis=0)[:,1].T
    data_c2 = pca_c2.fit(data.T)
    pcs_c2 = data_c2.components_
    pc2_evs.append(np.cumsum(pca_c2.explained_variance_ratio_))

    p_c1_true = []
    p_c2_true = []
    for c in range(fMRI_data[0][0].shape[1]):
        p_c1_true.append(np.dot(pcs_c1, np.mean(fMRI_data[r], axis=0)[:,c].T))
        p_c2_true.append(np.dot(pcs_c2, np.mean(fMRI_data[r], axis=0)[:,c].T))

    p_c1s = []
    p_c2s = []
    for repeat in range(repetition):
        
        beta_pcas = beta_perm_memory[r][repeat]

        pca_c1 = PCA(n_components=pc_num[r])
        data = np.mean(beta_pcas, axis=0)[:,0].T
        data_c1 = pca_c1.fit(data.T)
        pcs_c1 = data_c1.components_

        pca_c2 = PCA(n_components=pc_num[r])
        data = np.mean(beta_pcas, axis=0)[:,1].T
        data_c2 = pca_c2.fit(data.T)
        pcs_c2 = data_c2.components_

        c1s = []
        c2s = []
        for c in range(fMRI_data[0][0].shape[1]):
            c1s.append(np.dot(pcs_c1, np.mean(fMRI_data[r], axis=0)[:,c].T))
            c2s.append(np.dot(pcs_c2, np.mean(fMRI_data[r], axis=0)[:,c].T))
        p_c1s.append(c1s)
        p_c2s.append(c2s)

    p_c1_trues.append(np.array(p_c1_true))
    p_c2_trues.append(np.array(p_c2_true))
    p_c1_perms.append(np.array(p_c1s))
    p_c2_perms.append(np.array(p_c2s))

In [26]:
# novelty_co_space_novelty_true = p_c1_trues
# novelty_val_space_novelty_true = p_c2_trues
# novelty_co_space_novelty_perm = p_c1_perms
# novelty_val_space_novelty_perm = p_c2_perms

# novelty_co_space_memory_true = p_c1_trues
# novelty_val_space_memory_true = p_c2_trues
# novelty_co_space_memory_perm = p_c1_perms
# novelty_val_space_memory_perm = p_c2_perms

memory_space_memory_true = p_c1_trues
memory_space_memory_perm = p_c1_perms

# memory_space_novelty_true = p_c1_trues
# memory_space_novelty_perm = p_c1_perms

# recall_start_space_recall_true = p_c1_trues
# recall_start_space_recall_perm = p_c1_perms

# recall_end_space_recall_true = p_c1_trues
# recall_end_space_recall_perm = p_c1_perms